## Data discovery: Load and query Yellow Taxi data
> Download the dataset from [the official TLC Trip Record Data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

### This cell only shows how to document code
```python
# Load file
local_file = 'datasets/your-downloaded-from-TLC-taxis-file-here.parquet'

# Show data
spark.read.parquet(local_file).show()
```

In [13]:
from pyspark.sql import SparkSession

### What is master(local N)?
The --master option specifies the master URL for a distributed cluster, or local to run locally with one thread, or local[N] to run locally with N threads.

<b>Source</b>: See Spark [docs here](spark.apache.org/docs/latest). See all [options here](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls)

In [14]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

In [15]:
# Read taxi data
local_file = '../Sources/yellow_tripdata_2023-01.parquet'
df = spark.read.parquet(local_file)

In [16]:
# DF is like a relation table in memory. Let's see the columns
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [17]:
# Query sample:
df2 = df.select('VendorID','total_amount').where('total_amount > 1')

In [18]:
# Query sample:
# df.select('VendorID','total_amount').where('total_amount > 1').show(n=5)

In [19]:
# Query sample, using Spark SQL
df.createOrReplaceTempView('tbl_raw_yellow_taxis')

In [20]:
# SQL Statement
spark.sql('select VendorID, tpep_pickup_datetime, passenger_count from tbl_raw_yellow_taxis where total_amount > 1 and passenger_count > 2').show(n=5)

+--------+--------------------+---------------+
|VendorID|tpep_pickup_datetime|passenger_count|
+--------+--------------------+---------------+
|       1| 2023-01-01 07:43:37|            4.0|
|       1| 2023-01-01 07:03:36|            3.0|
|       1| 2023-01-01 07:21:49|            4.0|
|       2| 2023-01-01 07:27:16|            4.0|
|       2| 2023-01-01 07:15:13|            5.0|
+--------+--------------------+---------------+
only showing top 5 rows



In [21]:
# Stop the session
spark.stop()